<a href="https://colab.research.google.com/github/03hoshi25gtns80github/HuBERT_qiita/blob/main/HuBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchaudio transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torchaudio
import librosa
import soundfile as sf
from transformers import AutoFeatureExtractor, AutoModel

# wavファイルの読み込み（wav形式の音声データを読みこみます）
raw_speech, sr = sf.read("/content/drive/MyDrive/audio.wav")

# サンプリングレートの変換
if sr != 16000:
    raw_speech_16kHz = librosa.resample(raw_speech, orig_sr=sr, target_sr=16000)
else:
    raw_speech_16kHz = raw_speech

# モデルと特徴量抽出器の準備
model_name = "rinna/japanese-hubert-base"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()

# 特徴量の抽出（モデルと形式を表すために転置してます）
inputs = feature_extractor(raw_speech_16kHz, sampling_rate=16000, return_tensors="pt")
inputs['input_values'] = inputs['input_values'].transpose(0, 1)

# 入力データの形状を確認
print(f"入力のテンソルの形式: {inputs['input_values'].shape}")

outputs = model(**inputs)

# 埋め込み表現の取得
embeddings = outputs.last_hidden_state
print(f"出力の形式(batch_size, sequence_length, hidden_size): {embeddings.size()}")

Some weights of the model checkpoint at rinna/japanese-hubert-base were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertModel were not initialized from the model checkpoint at rinna/japanese-hubert-base and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

入力のテンソルの形式: torch.Size([1, 308700])
出力の形式(batch_size, sequence_length, hidden_size): torch.Size([1, 964, 768])
